# 전처리에서 일시 멈춤

# 1. 모듈

In [ ]:
from transformers.utils import default_cache_path

print(default_cache_path)

C:\Users\user\.cache\huggingface\hub


In [1]:
# 실행 전 PyTorch 캐시 비우기
import torch

torch.cuda.empty_cache()

In [2]:
import os, json, re
import pandas as pd
from tqdm import tqdm
from typing import List
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from PyPDF2 import PdfReader
from torch.utils.data import DataLoader

# GPU 확인용 코드
# CPU 사용시 느림 현상으로 멈춤 => GPU 사용
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 4070


# 2. 데이터 불러오기
## 2_1 파인튜닝/학습용 데이터

In [3]:
# 파일 경로
base_dir = "../dataset/"

# 파인튜닝/학습용 데이터
df_train_1 = pd.read_csv(base_dir + "1200_v1.csv")
df_train_2 = pd.read_csv(base_dir + "final_v7.csv")

# 확인 (처음 2개 row 출력)
print("파인튜닝 데이터셋1 (1200_clean):")
print(df_train_1.head(2), "\n")
print("=" * 100)
print("파인튜닝 데이터셋2 (final_v4):")
print(df_train_2.head(2), "\n")

파인튜닝 데이터셋1 (1200_clean):
  label                                               text
0    건선  지난 몇 주 동안 팔, 다리, 몸통에 피부 발진이 생겼어요. 빨갛고 가려운데, 마른...
1    건선  제 피부가 벗겨지고 있어요, 특히 무릎, 팔꿈치, 그리고 두피 쪽이요. 이 벗겨짐이... 

파인튜닝 데이터셋2 (final_v4):
      label                                               text
0      알레르기  저는 천식을 앓고 있어서 쌕쌕거리는 소리와 호흡 곤란이 있습니다. 또한 열이 나고 ...
1  조기 난소 부전  원발성 난소 부전의 징후와 증상은 폐경이나 에스트로겐 결핍과 유사합니다. 이러한 증... 



## 2_2 벡터DB용 데이터
### 2_2_1 클로링한 데이터

In [4]:
# 벡터DB용 데이터(클로링한 데이터)
df_vector_1 = pd.read_csv(base_dir + "snu_health_more_cleaned.csv")
df_vector_2 = pd.read_csv(base_dir + "amc_disease_more.csv")

# 확인 (처음 2개 row 출력)
print("벡터DB 데이터셋1 (snu_health_more_cleaned):")
print(df_vector_1.head(2), "\n")
print("=" * 100)
print("벡터DB 데이터셋2 (amc_disease_more):")
print(df_vector_2.head(2), "\n")

벡터DB 데이터셋1 (snu_health_more_cleaned):
                      병명                                                 정의  \
0  가랑이통증 [Perineal pain]  넓게는 외음부-여성의 바깥 생식기관-불두덩(치골구), 대음순, 소음순, 음핵, 처녀...   
1    가성근시 [pseudomyopia]  근시란 먼 곳을 볼 때 눈으로 들어간 빛의 초점이 망막보다 앞에 맺히는 것이다. 가...   

                                                  원인  \
0  요로감염이나 항문 농양 등의 감염, 해당 부위의 양성 또는 악성 종양, 관련 부위의...   
1  근거리 물체를 볼 때 수정체를 감싼 모양체 근육이 수축돼 수정체가 두꺼워지고 먼 거...   

                                                  증상  \
0  원인에 따라 증상에 다소 차이가 있을 수 있다. *요로감염: 배뇨통 및 빈뇨, 절박...   
1  원거리 시력저하가 나타나고 과도한 모양체 근육의 수축으로 인해서 눈이 피로하고 안구...   

                                               진단/검사  \
0                                                NaN   
1  가성근시는 일반적인 시력검사로는 알기 어렵다. 안과에서의 굴절 검사(현성 굴절 검사...   

                                                  치료          진료과  
0  원인에 따른 치료가 기본으로 감염일 경우 항생제, 농양일 경우 절개와 배농 등으로 ...         산부인과  
1  먼저 약물 치료 전 가성근시를 유발시킬 수 있는 생활조건에 대해 확인한다. 대부분 ...  안과, 정신건강의학과   

벡터DB 데이터셋2 

### 2_2_2 AI_HUB의 QA데이터
medicaL_legal_corpus/Validation/02.라벨링 데이터 불러오기

In [5]:
valid_label_dir = "../dataset/medical_knowledge_QA/Validation/02.라벨링데이터"
valid_qa_list = []

for filename in os.listdir(valid_label_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(valid_label_dir, filename)
        with open(file_path, encoding="utf-8-sig") as f:
            qa = json.load(f)
        valid_qa_list.append(qa)

print("Validation QA 샘플 수:", len(valid_qa_list))
if valid_qa_list:
    print("첫 번째 샘플:", valid_qa_list[0])
else:
    print("샘플 없음")

Validation QA 샘플 수: 1920
첫 번째 샘플: {'qa_id': 1103, 'domain': 17, 'q_type': 1, 'question': '25세 남자가 피로와 쉽게 멍이 드는 증상으로 내원하였다. 최근 몇 주간 무력감이 점점 심해졌으며, 약간의 운동에도 숨이 차다고 한다. 양치하다가 잇몸에서 피가 나는 경우도 잦다. 혈압 110/70mmHg, 맥박 75회/분, 호흡 18회/분, 체온 38.3°C이다. 신체 진찰 시 결막은 창백하고 공막에 황달은 없었다. 간과 지라는 촉진되지 않았고, 가슴 X선 촬영도 정상이다. 검사 결과는 다음과 같다. 다음으로 시행해야 할 검사는?  \n혈액 검사 결과: 혈색소 8.0g/dL, 백혈구 2,000/mm³, 혈소판 50,000/mm³, C-반응단백질 31mg/L (참고치, <10) \n\n1) 심초음파  \n2) 복부 및 골반 컴퓨터단층촬영  \n3) 철 결핍 검사  \n4) 골수흡인 및 생검  \n5) 복부초음파 검사', 'answer': '4) 골수흡인 및 생검'}


# 3 파인튜닝
## 3_1. 데이터 저장 (CSV → jsonl 변환)

In [6]:
# 데이터셋 1
df_train_1.rename(columns={"label": "instruction", "text": "output"}, inplace=True)
df_train_1.to_json(
    "1200_v1_train.jsonl", orient="records", lines=True, force_ascii=False
)

# 데이터셋 2
df_train_2.rename(columns={"label": "instruction", "text": "output"}, inplace=True)
df_train_2.to_json(
    "final_v6_train.jsonl", orient="records", lines=True, force_ascii=False
)

## 3_2. 파인튜닝 코드

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
import torch

# 1. 모델 및 토크나이저 로드
# nlpai-lab/KULLM3
HF_TOKEN = ""  # 허깅페이스 개인 키
model_name = "nlpai-lab/kullm-polyglot-5.8b-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    device_map="auto",
    torch_dtype="auto",
    offload_folder="./offload",
)

# 2. 데이터셋 불러오기 & 전처리
dataset1 = load_dataset("json", data_files="1200_v1_train.jsonl", split="train")


def format_prompt(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Answer:\n{example['output']}"
    }


dataset1 = dataset1.map(format_prompt)


def tokenize_function(example):
    tokenized = tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=256
    )
    # labels 추가 (input_ids 복사)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


tokenized_dataset1 = dataset1.map(tokenize_function, batched=True)

# 텐서만 남기고 DataLoader 사용
tokenized_dataset1.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)
loader1 = DataLoader(tokenized_dataset1, batch_size=1, shuffle=True)

# 3. 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 4. 파인튜닝 루프
model.train()
for epoch in range(2):
    for batch in loader1:
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(loss.item())

In [ ]:
# 두 번째 데이터셋 불러오기 & 전처리
dataset2 = load_dataset("json", data_files="final_v6_train.jsonl", split="train")


def format_prompt(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Answer:\n{example['output']}"
    }


dataset2 = dataset2.map(format_prompt)


def tokenize_function(example):
    tokenized = tokenizer(
        example["text"], truncation=True, padding="max_length", max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


tokenized_dataset2 = dataset2.map(tokenize_function, batched=True)
tokenized_dataset2.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)
loader2 = DataLoader(tokenized_dataset2, batch_size=1, shuffle=True)

# 3. 옵티마이저 준비
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 4. 파인튜닝 루프 (dataset2)
model.train()
for epoch in range(2):
    for batch in loader2:
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(loss.item())

# 4. 벡터DB 구축 (청크 + 임베딩 + 저장)
(1) 데이터 프리프로세싱:

snu_health_more_cleaned.csv, amc_disease_more.csv 등에서 텍스트 추출

(2) 텍스트 청크(chunk) 단위로 자르기:

(예시) RecursiveCharacterTextSplitter 등 사용

길이(예: 300~1000자)와 오버랩(겹침) 설정

(3) 임베딩 생성:

SBERT/KoSBERT 등 임베딩 모델로 청크별 임베딩 벡터 생성

(4) 벡터DB 저장:

FAISS, Qdrant 등 선택

# 5. RAG 파이프라인 구축
(1) 모델 로딩:

파인튜닝된 Llama-2-7b-chat-hf 모델 (허가 후 다운)

(2) 벡터DB 로딩:

저장된 FAISS 등 벡터DB 로드

(3) 검색 + 생성 연결:

사용자의 질문 → 임베딩 → 벡터DB에서 유사 문서 검색

찾은 컨텍스트 + 질문을 모델 프롬프트로 전달

모델이 답변 생성

# 6.질문 입력 → 답변 출력
- 간단한 CLI, Jupyter, 또는 ipywidgets UI 활용 가능
- 실시간 질문 입력 → RAG 파이프라인 → 답변



# 샘플 코드 워크플로우 (요약)

In [ ]:
# 1. 청크+임베딩+벡터DB 저장 (한 번만)
# splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
# documents = splitter.create_documents(your_texts)
# embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
# db = FAISS.from_documents(documents, embedding=embedding_model)
# db.save_local("your_vector_db_folder")

# 2. 파인튜닝된 LLM과 벡터DB 로드, RAG 체인 연결
# model = AutoModelForCausalLM.from_pretrained(...)
# tokenizer = AutoTokenizer.from_pretrained(...)
# llm_pipeline = pipeline(...)
# llm = HuggingFacePipeline(pipeline=llm_pipeline)
# db = FAISS.load_local("your_vector_db_folder", embedding_model)
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":5})

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
# )

# 3. 질문 입력 → 답변 출력
# query = "질문 입력"
# result = qa_chain({"query": query})
# print(result["result"])
# print(result["source_documents"])